In [1]:
import pymongo
print(pymongo.__version__)

4.7.3


### Connect

In [3]:
#server
client = pymongo.MongoClient("mongodb://localhost:27017/")
#database
mydb = client["Engines"]
#collection
carinfo = mydb.carInformation
engineingo = mydb.engineInformation

In [5]:
for record in carinfo.find():
    print(record)

{'_id': ObjectId('66763bc9fc5299048e577847'), 'model': 'covette', 'yom': 2007, 'origin': 'USA', 'engine': {'engine_name': 'LS3', 'manufacture': 'GM', 'capacity': '6.2L', 'horse_power': 425}}
{'_id': ObjectId('66763bc9fc5299048e577848'), 'model': 'supra', 'yom': 2000, 'origin': 'Japan', 'engine': {'engine_name': '2JZ', 'manufacture': 'Toyota', 'capacity': '3.0L', 'horse_power': 350}}
{'_id': ObjectId('66763bc9fc5299048e577849'), 'model': 'mustang', 'yom': 2013, 'origin': 'USA', 'engine': {'engine_name': 'Cayote', 'manufacture': 'Ford', 'capacity': '5.0L', 'horse_power': 475}}
{'_id': ObjectId('66763bc9fc5299048e57784a'), 'model': 'f250', 'yom': 2017, 'origin': 'USA', 'engine': {'engine_name': 'Godzilla', 'manufacture': 'Ford', 'capacity': '7.0L', 'horse_power': 525}}


### Update

In [24]:
###Update One
##$set
#update existing filed
carinfo.update_one(
    {"model":"covette"},        #filter
    {"$set":{"engine.horse_power":455}}                  #update
    )
for record in carinfo.find({'model':'covette'}):
    print(f"updated record : {record}\n")

#create new filed
carinfo.update_one(
    {"model":"f250"},
    {"$set":{"engine_status":"running lean"}}
)
for record in carinfo.find({'model':'f250'}):
    print(f"new field updated record : {record}\n")
    
#$unset
carinfo.update_one(
    {"model":"f250"},
    {"$unset":{"engine_status":""}}
)
for record in carinfo.find({'model':'f250'}):
    print(f"new field deleted record : {record}\n")

#$inc
#increment
carinfo.update_one(
    {"model":"f250"},
    {"$inc":{"engine.horse_power":10}}
)
for record in carinfo.find({'model':'f250'}):
    print(f"incremental record : {record}\n")
#decrement
carinfo.update_one(
    {"model":"f250"},
    {"$inc":{"engine.horse_power":-10}}
)
for record in carinfo.find({'model':'f250'}):
    print(f"incremental record : {record}\n")

updated record : {'_id': ObjectId('66763bc9fc5299048e577847'), 'model': 'covette', 'yom': 2007, 'origin': 'USA', 'engine': {'engine_name': 'LS3', 'manufacture': 'GM', 'capacity': '6.2L', 'horse_power': 455}}

new field updated record : {'_id': ObjectId('66763bc9fc5299048e57784a'), 'model': 'f250', 'yom': 2017, 'origin': 'USA', 'engine': {'engine_name': 'Godzilla', 'manufacture': 'Ford', 'capacity': '7.0L', 'horse_power': 545}, 'engine_status': 'running lean'}

new field deleted record : {'_id': ObjectId('66763bc9fc5299048e57784a'), 'model': 'f250', 'yom': 2017, 'origin': 'USA', 'engine': {'engine_name': 'Godzilla', 'manufacture': 'Ford', 'capacity': '7.0L', 'horse_power': 545}}

incremental record : {'_id': ObjectId('66763bc9fc5299048e57784a'), 'model': 'f250', 'yom': 2017, 'origin': 'USA', 'engine': {'engine_name': 'Godzilla', 'manufacture': 'Ford', 'capacity': '7.0L', 'horse_power': 555}}

incremental record : {'_id': ObjectId('66763bc9fc5299048e57784a'), 'model': 'f250', 'yom': 2017

In [31]:
for record in carinfo.find():
    model = record['model']
    capacity = float(record['engine']['capacity'][:-1])
    carinfo.update_one(
        {'model':model},
        {'$set':{'engine.capacity':capacity}}
    )

In [37]:
##Update Many
##$set
carinfo.update_many(
    {'origin':'USA'},             #filter
    {"$set":{'domestic':True}}    #update
)
carinfo.update_many(
    {'origin':{"$ne":"USA"}},          #filter
    {"$set":{'domestic':False}}        #update
)

for record in carinfo.find():
    print(f"{record}\n")

{'_id': ObjectId('66763bc9fc5299048e577847'), 'model': 'covette', 'yom': 2007, 'origin': 'USA', 'engine': {'engine_name': 'LS3', 'manufacture': 'GM', 'capacity': 6.2, 'horse_power': 455}, 'domestic': True}

{'_id': ObjectId('66763bc9fc5299048e577848'), 'model': 'supra', 'yom': 2000, 'origin': 'Japan', 'engine': {'engine_name': '2JZ', 'manufacture': 'Toyota', 'capacity': 3.0, 'horse_power': 350}, 'domestic': False}

{'_id': ObjectId('66763bc9fc5299048e577849'), 'model': 'mustang', 'yom': 2013, 'origin': 'USA', 'engine': {'engine_name': 'Cayote', 'manufacture': 'Ford', 'capacity': 5.0, 'horse_power': 475}, 'domestic': True}

{'_id': ObjectId('66763bc9fc5299048e57784a'), 'model': 'f250', 'yom': 2017, 'origin': 'USA', 'engine': {'engine_name': 'Godzilla', 'manufacture': 'Ford', 'capacity': 7.0, 'horse_power': 545}, 'domestic': True}



### Replace

In [40]:
carinfo.replace_one(
    {'model':'f250'},
    {'model': 'f150',
     'yom': 2018,
     'origin': 'USA',
     'engine': {'engine_name': 'Godzilla',
                'manufacture': 'Ford',
                'capacity': 7.0,
                'horse_power': 545}
     , 'domestic': True}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### Delete

In [46]:
carinfo.insert_one({'model': 'RAM',
     'yom': 2018,
     'origin': 'USA',
     'engine': {'engine_name': 'Cummins',
                'manufacture': 'Dodge',
                'capacity': 7.0,
                'horse_power': 345}
     , 'domestic': True})

InsertOneResult(ObjectId('6677729b703215bd28758546'), acknowledged=True)

In [51]:
carinfo.delete_one(
    {"engine.manufacture":"Dodge"},     #filter
)
for record in carinfo.find({"engine.manufacture":"Dodge"}):
    print(record)

In [52]:
data = [
    {'model': 'Corolla',
     'yom': 2018,
     'origin': 'Japan',
     'engine': {'engine_name': '1NZ',
                'manufacture': "Toyota",
                'capacity': 1.5,
                'horse_power': 90}
     , 'domestic': False},
    {'model': 'Civic',
     'yom': 1999,
     'origin': 'Japan',
     'engine': {'engine_name': 'K20',
                'manufacture': 'Honda',
                'capacity': 2.0,
                'horse_power': 165}
     , 'domestic': False},
    {'model': 'Land Cruiser',
     'yom': 2012,
     'origin': 'Japan',
     'engine': {'engine_name': '1UFZ',
                'manufacture': 'Toyota',
                'capacity': 5.0,
                'horse_power': 365}
     , 'domestic': False}
]
carinfo.insert_many(data)
carinfo.delete_many(
    {"engine.capacity":{"$lt":2},"engine.horse_power":{"$lt":150}}
)
for recor in carinfo.find({"engine.capacity":{"$lt":2},"engine.horse_power":{"$lt":150}}):
    print(record)